# OpenSNP

## Packages and functions

In [1]:
import os
import sys
import pickle
import json
import subprocess
import pandas as pd
import numpy as np

import requests
from lxml import html

## Uploads

In [2]:
url = "https://opensnp.org/genotypes?page={}"

# Get the number of pages
page = requests.get(url.format(1))
tree = html.fromstring(page.content)
num_pages = int(tree.xpath('//div[@class="pagination"]/ul/li/a/text()')[-1])

# Get the number of genotypes per page
page = requests.get(url.format(1))
tree = html.fromstring(page.content)
num_genotypes = len(tree.xpath('//div[@class="genotype"]/div[@class="genotype-name"]/a/text()'))

# Get the number of genotypes
num_genotypes = num_pages * num_genotypes

# Get the number of users
page = requests.get(url.format(1))
tree = html.fromstring(page.content)
num_users = len(tree.xpath('//div[@class="genotype"]/div[@class="genotype-user"]/a/text()'))



IndexError: list index out of range

In [5]:
page.content

b'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8" http-equiv="X-UA-Compatible" content="IE=edge">\n    <title>openSNP | Listing all genotypings</title>\n    <link rel="shortcut icon" href="/favicon.ico" />\n    <meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />\n    <link rel="stylesheet" media="screen" href="/assets/application-103f393d66a70948281b7e5ee5659703b79fd7135f25a81ff020a9d69d493174.css" />\n    <script src="/assets/application-51914e357728cb5c966cbb431070efbf663f9e10e48f2eacf29fde995217a149.js"></script>\n    <meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="kZmTc7fkFUz+AT3v4OZGAEKXkYXZzcrW/WQK9Wxf9+PaWcvQqfD5nukYis6q9GNlxS1SvBbMGgEUEZTpXFXyRA==" />\n  </head>\n  <body>\n    <nav class="navbar-default navbar-menu">\n  <div class="container">\n    <div class="navbar-header">\n      <a class="navbar-brand navbar-menu__brand" href="/">openSNP</a>\n      <button t

In [6]:
url = "https://opensnp.org/genotypes?page={}"

all_files = []

# Get table of file information
req = requests.session()
for i in range(1, 400):
    page = req.get(url.format(i), verify=False)
    tree = html.fromstring(page.text)
    
    # Get file name
    file_names = [x.lstrip('../data/') for x in tree.xpath("//table[@class='table table-hover']//tr//td[5]/a/@href")]
    if len(file_names) == 0:
        break

    # Get file ID
    fileIDs = [x.lstrip() for x in tree.xpath("//table[@class='table table-hover']//td[2]//text()[1]")]
    
    # Get time uplodaded
    times = [x.lstrip() for x in tree.xpath("//table[@class='table table-hover']//td[3]//text()[1]")]

    # Get user ID
    nicknames = [x.lstrip() for x in tree.xpath("//table[@class='table table-hover']//td[1]//text()[1]") if not '\n' in x]

    assert (len(file_names) == len(fileIDs) == len(times) == len(nicknames))

    for j in range(0, len(file_names)):
        assert(fileIDs[j] == file_names[j].split('.')[2])
        all_files.append({
            'nickname': nicknames[j], \
            'userID': file_names[j].split('.')[0], \
            'fileID': fileIDs[j], \
            'filename': file_names[j], \
            'time': times[j], \
            'download': 'https://opensnp.org/data/' + file_names[j]
        })

/Users/clu/softwoare/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opensnp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/clu/softwoare/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opensnp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/clu/softwoare/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opensnp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/clu/softwoar

In [7]:
uploads = pd.DataFrame({
    'nickname': [x['nickname'] for x in all_files], \
    'userID': [x['userID'] for x in all_files], \
    'fileID': [x['fileID'] for x in all_files], \
    'filename': [x['filename'] for x in all_files], \
    'time': [x['time'] for x in all_files], \
    'download': [x['download'] for x in all_files]
})
uploads.to_csv('data/opensnp_uploads.csv', index=False)

In [8]:
uploads

,nickname,userID,fileID,filename,time,download
0,LauraM,12086,9953,12086.23andme.9953,24.09.2023 19:02,https://opensnp.org/data/12086.23andme.9953
1,Claudia,12082,9951,12082.23andme.9951,23.09.2023 12:13,https://opensnp.org/data/12082.23andme.9951
2,Claudia,12082,9950,12082.23andme.9950,23.09.2023 12:11,https://opensnp.org/data/12082.23andme.9950
3,Matuomala,12081,9949,12081.23andme.9949,22.09.2023 18:39,https://opensnp.org/data/12081.23andme.9949
4,guillaume,12080,9948,12080.ftdna-illumina.9948,22.09.2023 10:20,https://opensnp.org/data/12080.ftdna-illumina....
...,...,...,...,...,...,...
6850,Lb,14,6,14.23andme.6,30.09.2011 11:12,https://opensnp.org/data/14.23andme.6
6851,Nash Parovoz,6,5,6.23andme.5,30.09.2011 10:21,https://opensnp.org/data/6.23andme.5
6852,sagan,13,4,13.23andme.4,30.09.2011 07:48,https://opensnp.org/data/13.23andme.4
6853,Svan,10,3,10.23andme.3,29.09.2011 21:00,https://opensnp.org/data/10.23andme.3
